In [ ]:
import psycopg2
from postgres_utilities import connect_db, close_db
import pandas as pd
from io import StringIO

In [ ]:
try:
    conn = connect_db()
    # Read the train.csv file into a pandas dataframe, skipping bad lines
    df = pd.read_csv("./products.csv", on_bad_lines="skip")  
    output = StringIO()
    df_copy = df.copy()

    # Convert year to integer if it's not already
    df_copy['year'] = df_copy['year'].astype('Int64')

    # Replace NaN with None for proper NULL handling in PostgreSQL
    df_copy = df_copy.replace({pd.NA: None, pd.NaT: None})
    df_copy = df_copy.where(pd.notnull(df_copy), None)
    
    # Convert DataFrame to csv format in memory
    df_copy.to_csv(output, sep='\t', header=False, index=False, na_rep='\\N')
    output.seek(0)
    # Copy the data to the products table
    with conn.cursor() as cur:
        # Use COPY to insert data
        cur.copy_from(file=output,table='products',null='\\N')
    conn.commit()
except Exception as e:
    if conn is not None and not conn.closed:
        conn.rollback()
    err_msg = f"An error occurred and the transaction was rolled back: {e}"
    raise Exception(err_msg)
finally:
    close_db(conn)